In [1]:
import scipy.io
import numpy as np
from scipy.stats import beta, multivariate_normal, poisson
from scipy.linalg import inv

np.random.seed(0)

# for matlab code to run in vscode pip install https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html
data = scipy.io.loadmat("/home/ari/Desktop/Bayesian/BCL/chunk_learner_python/input_2001_Exp1.mat")

X = data["X"]
V = data["V"]
X_test = data["Xtest"]
V_test = data["Vtest"]

N, T = X.shape

#params
lda = 0.999
eps = 0.01
sigmaU = 3
sigmaV = 3
sigmaC = 0.1
phi = 4
bAlpha = 1.0
bBeta = 1.0
alpha = 0.1
trainingLength = 6
Kmax = 1
wburn = 5
wsample = 10
stepNo = 10
burnIn = 0

def calc_pv_trial(x, v, y, Z, sigmaU, sigmaV, phi):
    N, K = Z.shape
    ZAct = Z * np.tile(y.flatten(), (N, 1))

    phi = phi * ZAct
    zc = np.logical_and(phi, x)

    xAct = np.where(x == 1)[0] + 1
    xActNo = len(xAct)
    A = phi / (1 + np.tile(np.sum(phi, axis=1, keepdims=True), (1,K)))

    sSq = (sigmaV ** 2 / (1 + np.sum(phi, axis=1)))
    ptAct  = np.where(np.sum(ZAct * (np.tile(x.flatten(), (K,1)).T), axis=0))[0] + 1 # to match matlab indexing
    ptNo = len(ptAct)

    SigmaInv = np.zeros((ptNo, ptNo, len(xAct)))
    omegaBU = np.zeros((N, N))
    diag_values = np.diag(sSq[xAct-1])
    omega = 1 / sigmaV**4 * diag_values

    availLinks = []
    availLinksRow = []
    availLinksCol = []

    if np.sum(Z) != 0:
        availLinksRow, availLinksCol = np.where(Z) # col, row = 1, 0 in matlab 2, 1
        availLinks = np.column_stack((availLinksRow+1, availLinksCol+1)) # to match matlab indexing of Z
        availLinkNo = availLinks.shape[0]
        SigmaCtllhInv = np.zeros((availLinkNo, availLinkNo))
        wt = np.zeros((availLinkNo, 2))
    else:
        availLinkNo = 0
        SigmaCtllhInv = 0
        wt = 0

    if ptNo > 0:
        for xi in range(1, len(xAct) + 1):
            row_indices = xAct[xi-1]-1
            col_indices = ptAct-1
            array = A[np.ix_([row_indices], col_indices)]
            array_T = array.T
            SigmaInv[:, :, xi-1] = (array * array_T) / sSq[xAct[xi-1]-1]
    
        SigmaCVInv = np.sum(SigmaInv, axis=2)
        SigmaCInv = SigmaCVInv + 1 / (sigmaU**2) * np.eye(ptNo)
        SigmaC = np.linalg.inv(SigmaCInv)

        if SigmaC.size == 1:
            ptAct = ptAct[0]
            A_submatrix = A[xAct - 1, ptAct - 1]
            A_submatrix = A_submatrix.reshape(-1, 1)
            omega = omega - (1 / sigmaV**4) * (A_submatrix @ (SigmaC @ A_submatrix.T))
        else:
            A_submatrix = A[xAct - 1, :][:, ptAct - 1]
            omega = omega - 1 / sigmaV**4 * (A_submatrix @ SigmaC @ A_submatrix.T)

        omegaBU = np.zeros((N,N))
        omegaBU[np.ix_(xAct-1, xAct-1)] = omega

        for i in range(1, availLinkNo+1):
            for j in range(1, availLinkNo+1):
                if x[availLinks[i-1, 0]-1] == 1 and x[availLinks[j-1, 0]-1] == 1:
                    SigmaCtllhInv[i-1, j-1] = (omegaBU[availLinks[i-1, 0]-1, availLinks[j-1, 0]-1] * 
                                               phi[availLinks[i-1, 0]-1, availLinks[i-1, 1]-1] * 
                                               phi[availLinks[j-1, 0]-1, availLinks[j-1, 1]-1])

            vnan = np.nan_to_num(v)
            #v array shape in the beginning is (12,144,2) due to matlab slicing, using python's (12,2)
            first_term = phi[availLinks[i-1,0]-1, availLinks[i-1,1]-1]
            second_term = omegaBU[availLinks[i-1,0]-1,:]
            result1 = first_term * second_term
            result1 = result1.reshape(1,-1)

            third_term = np.sum(phi, axis=1) + 1
            fourth_term_wt1 = vnan[:,0]
            fourth_term_wt2 = vnan[:,1]

            result2_wt1 = third_term * fourth_term_wt1
            result2_wt2 = third_term * fourth_term_wt2
            final_result_wt1 = np.dot(result1, result2_wt1)
            final_result_wt2 = np.dot(result1, result2_wt2)

            wt[i-1, 0] = final_result_wt1
            wt[i-1, 1] = final_result_wt2

    else:
        SigmaC = 1

    if type(SigmaC) == int:
        det_SigmaC = 1
    else:
        det_SigmaC = np.linalg.det(SigmaC)

    Psit = np.zeros((1, 1, 2))
    vp = (1 + np.sum(phi, axis=1)) * v[:, 0]
    vpnan = np.nan_to_num(vp)
    Psit[:,:,0] = (2*np.pi)**(-xActNo/2) * sigmaU**(-ptNo) * 1/(np.sqrt(np.prod(sSq[xAct-1]))) * det_SigmaC**(0.5) * np.exp(-0.5 * np.dot(vpnan.T, np.dot(omegaBU, vpnan)))
    
    vp = (1 + np.sum(phi, axis=1)) * v[:, 1]
    vpnan = np.nan_to_num(vp)
    Psit[:,:,1] = (2*np.pi)**(-xActNo/2) * sigmaU**(-ptNo) * 1/(np.sqrt(np.prod(sSq[xAct-1]))) * det_SigmaC**(0.5) * np.exp(-0.5 * np.dot(vpnan.T, np.dot(omegaBU, vpnan)))
    
    return SigmaCtllhInv, wt, Psit, zc

def calc_pv_training(x, v, y, Z, sigmaU, sigmaV, phi, sigmaC, *args):
    nargs = len(args)
    dims = args[0] if nargs > 0 else 0
    logOutput = args[1] if nargs > 1 else 0

    N, T = x.shape
    K = Z.shape[1]
    availLinkNo = int(np.sum(Z))
    zc = np.zeros((N, K), dtype=bool)
    zl = Z.astype(bool)
    SigmaCtllhInv = np.zeros((availLinkNo, availLinkNo, T))
    wt = np.zeros((availLinkNo, T, 2))
    Psit = np.zeros((1, T, 2))

    for t in range(1, T+1):

        SigmaCtllhInv[:, :, t-1], wt[:, t-1, :], Psit[:, t-1, :], zct = calc_pv_trial(x[:, t-1:t], v[:, t-1, :] , y[:, t-1:t], Z, sigmaU, sigmaV, phi)
        zc = np.bitwise_or(zc, zct)
       
    SigmaCTllhInv = np.sum(SigmaCtllhInv, axis=2)
    SigmaCTpostInv = SigmaCTllhInv + np.diag(np.ones(availLinkNo)) / (sigmaC ** 2)
    SigmaCTpost = np.linalg.inv(SigmaCTpostInv)
    muCTpost = np.dot(SigmaCTpost, np.sum(wt[:, :, 0], axis=1))
    muCTpost = muCTpost[:, np.newaxis]
    muCTpost = np.concatenate((muCTpost, np.dot(SigmaCTpost, np.sum(wt[:,:,1], axis=1))[:, np.newaxis]), axis=1)
   
    muCTpostX = muCTpost[:, 0]
    muCTpostY = muCTpost[:, 1]

    #matches the result in matlab mvnormdfln
    if np.sum(zc):
        zc_zl = zc[zl]
        nfx = multivariate_normal.logpdf(muCTpostX[zc_zl], mean=np.zeros(np.sum(zc)), cov=inv(SigmaCTpostInv[zc_zl, :][:, zc_zl]))
        nfy = multivariate_normal.logpdf(muCTpostY[zc_zl], mean=np.zeros(np.sum(zc)), cov=inv(SigmaCTpostInv[zc_zl, :][:, zc_zl]))
    else:
        nfx = 0
        nfy = 0

    if logOutput == 0:
        p1 = np.prod(Psit[:, :, 0]) * (2 * np.pi) ** (-np.sum(zc) / 2) * sigmaC ** (-np.sum(zc)) / np.exp(nfx)
        p2 = np.prod(Psit[:, :, 1]) * (2 * np.pi) ** (-np.sum(zc) / 2) * sigmaC ** (-np.sum(zc)) / np.exp(nfy)

        if dims == 0:
            p = p1 * p2
        elif dims == 1:
            p = p1
        elif dims == 2:
            p = p2
        else:
            raise ValueError('calc_pv_training: wrong setting for optional argument dims')
    else:
        lp1 = np.sum(np.log(Psit[:, :, 0])) + np.log((2 * np.pi) ** (-np.sum(zc) / 2) * sigmaC ** (-np.sum(zc))) - nfx
        lp2 = np.sum(np.log(Psit[:, :, 1])) + np.log((2 * np.pi) ** (-np.sum(zc) / 2) * sigmaC ** (-np.sum(zc))) - nfy

        if dims == 0:
            p = lp1 + lp2
        elif dims == 1:
            p = lp1
        elif dims == 2:
            p = lp2
        else:
            raise ValueError('calc_pv_training: wrong setting for optional argument dims')

    return p, muCTpost, SigmaCTpostInv, SigmaCTpost

def calc_px_training(X, Y, Z, lda, eps, *args):
    nargs = len(args)
    isLog = args[0] if nargs > 0 else 0

    if Z.ndim == 1:  #cases (1,)  (2,)  (3,) (4,)
        Z = Z.reshape(-1, 1) 
        pp = (1 - ((1 - lda) ** (Z.T @ Y)) * (1 - eps)) * X + (((1 - lda) ** (Z.T @ Y)) * (1 - eps)) * (1 - X)
    else:  #cases (12,1) (12,3)
        pp = (1 - ((1 - lda) ** (Z @ Y)) * (1 - eps)) * X + (((1 - lda) ** (Z @ Y)) * (1 - eps)) * (1 - X)

    if isLog == 0:
        p = np.prod(pp)
    else:
        pp = np.log(pp)
        p = np.sum(pp)

    return p

def wood_make_gibbs_y_spatial(Y, X, V, Z, R, lda, eps, sigmaU, sigmaV, phi, sigmaC):

    K, T = Y.shape
    Ynew = Y.copy()

    lpv_y = np.zeros((1, 2))
    lpx_y = np.zeros((1, 2))
    lpv_y_new, _, _, _ = calc_pv_training(X, V, Ynew, Z, sigmaU, sigmaV, phi, sigmaC, 0, 1)
    lpx_y_new = calc_px_training(X[:, 0:1], Ynew[:, 0:1], Z, lda, eps, 1)

    for t in range(1, T + 1):
        for k in range(1, K + 1):
            lpv_y[0, Ynew[k-1, t-1]] = lpv_y_new
            lpx_y[0, Ynew[k-1, t-1]] = lpx_y_new

            Ynew[k-1, t-1] = 1 - Ynew[k-1, t-1]

            lpv_y[0, Ynew[k-1, t-1]], _, _, _ = calc_pv_training(X, V, Ynew, Z, sigmaU, sigmaV, phi, sigmaC, 0, 1)
            lpx_y[0, Ynew[k-1, t-1]] = calc_px_training(X[:, t-1:t], Ynew[:, t-1:t], Z, lda, eps, 1)

            p = R[0, k-1]

            lPYk1_ZXYmk = np.log(p) + lpx_y[0, 1] + lpv_y[0, 1]
            lPYk0_ZXYmk = np.log(1 - p) + lpx_y[0, 0] + lpv_y[0, 0]

            rt = np.exp(lPYk1_ZXYmk - lPYk0_ZXYmk)

            if rt == np.inf:
                PYk_ZXYmk = 1
            else:
                PYk_ZXYmk = rt / (rt + 1)

            Ynew[k-1, t-1] = PYk_ZXYmk > np.random.rand(1,1)

            lpv_y_new = lpv_y[0, Ynew[k-1, t-1]]
            lpx_y_new = lpx_y[0, Ynew[k-1, t-1]]

    return Ynew

def wood_make_gibbs_z_spatial(Y, X, V, Z, R, lda, eps, sigmaU, sigmaV, phi, sigmaC, bAlpha, bBeta, alpha, Kmax, wburn, wsample):
    
    N, K = Z.shape
    T = X.shape[1]

    Znew = Z.copy()
    lpv_y = np.zeros((1,2))
    lpx_y = np.zeros((1,2))

    wall = wburn + wsample

    for n in range(1, N + 1):
        Znew = Z.copy()
        lpv_y_new, _, _, _ = calc_pv_training(X, V, Y, Znew, sigmaU, sigmaV, phi, sigmaC, 0, 1)
        lpx_y_new = calc_px_training(X[n-1, :], Y, Znew[n-1, :], lda, eps, 1)

        for k in range(1, K + 1):
            if n - 1 < N - 1:
                m_mnk = np.sum(Znew[np.r_[0:n-1, n:N], k-1])
            else:
                m_mnk = np.sum(Znew[0:n-1, k-1])

            th_k = m_mnk / N

            if m_mnk > 0:
                lpv_y[0, int(Znew[n-1, k-1])] = lpv_y_new
                lpx_y[0, int(Znew[n-1, k-1])] = lpx_y_new

                Znew[n-1, k-1] = 1 - Znew[n-1, k-1]

                lpv_y[0, int(Znew[n-1, k-1])], _, _, _ = calc_pv_training(X, V, Y, Znew, sigmaU, sigmaV, phi, sigmaC, 0, 1)
                lpx_y[0, int(Znew[n-1, k-1])] = calc_px_training(X[n-1, :], Y, Znew[n-1, :], lda, eps, 1)

                lPznk1_XZmnkY = np.log(th_k) + lpx_y[0, 1] + lpv_y[0, 1]
                lPznk0_XZmnkY = np.log(1 - th_k) + lpx_y[0, 0] + lpv_y[0, 0]
                
                rt = np.exp(lPznk1_XZmnkY - lPznk0_XZmnkY)

                if rt == np.inf:
                    Pznk_XZmnkY = 1
                else:
                    Pznk_XZmnkY = rt / (rt + 1)
            else:
                Pznk_XZmnkY = 0

            uni = np.random.rand()

            Znew[n-1, k-1] = Pznk_XZmnkY > uni

            lpv_y_new = lpv_y[0, int(Znew[n-1, k-1])]
            lpx_y_new = lpx_y[0, int(Znew[n-1, k-1])]

        Z = Znew.copy()
        for k in range(1, K + 1):
            R[0, k-1] = beta.rvs((bAlpha + np.sum(Y[k-1, :])), (bBeta + T - np.sum(Y[k-1, :])))

        PKnew_XZY = np.zeros((1, Kmax + 1))
        PKnew_XZY_an = np.zeros((1, Kmax + 1))
        PKnew_XZY_5 = np.zeros((1, Kmax + 1))

        Ysample = {}
        Rsample = {}

        for Knew in range(Kmax + 1):
            lpy_xz = np.zeros((1, wsample))
            ZAct = np.hstack((Z, np.zeros((N, Knew))))
            if Knew > 0:
                ZAct[n - 1, K:K + Knew] = 1
            RAdd = beta.rvs(bAlpha, bBeta, size=(1,Knew))
            RAct = np.hstack([R, RAdd])
            if RAdd.shape == (1,0):
                RAdd = 0

            # rng result is different here, in matlab [1,0], in python [1,1]
            YAct = np.vstack([Y, (np.random.rand(Knew, T) < np.tile(RAdd, (Knew, 1))).astype(int)])

            lpv_y_act = np.zeros((1,2))
            lpx_y_act = np.zeros((1,2))
            
            lpv_y_new, _, _, _ = calc_pv_training(X, V, YAct, ZAct, sigmaU, sigmaV, phi, sigmaC, 0, 1)
            lpx_y_new = calc_px_training(X[n-1, :], YAct, ZAct[n-1, :], lda, eps, 1)

            if Knew > 0:
                lpy_v = np.zeros(wsample)
                lpy_x = np.zeros(wsample)
                lpy_xv = np.zeros(wsample)
                for wi in range(1, wall+1):
                    for t in range(1, T + 1):
                        for k in range(1, Knew + 1):
                            lpv_y_act[0, int(YAct[K + k-1, t-1])] = lpv_y_new
                            lpx_y_act[0, int(YAct[K + k-1, t-1])] = lpx_y_new

                            YAct[K + k-1, t-1] = 1 - YAct[K + k-1, t-1]
                            lpv_y_act[0, int(YAct[K + k-1, t-1])], _, _, _ = calc_pv_training(X, V, YAct, ZAct, sigmaU, sigmaV, phi, sigmaC, 0, 1)
                            lpx_y_act[0, int(YAct[K + k-1, t-1])] = calc_px_training(X[n-1, :], YAct, ZAct[n-1, :], lda, eps, 1)
                            p = RAct[0, (K + k-1)]
                            lPYk1_ZXYmk = np.log(p) + lpx_y_act[0,1] + lpv_y_act[0,1]
                            lPYk0_ZXYmk = np.log(1 - p) + lpx_y_act[0,0] + lpv_y_act[0,0]

                            rt = np.exp(lPYk1_ZXYmk - lPYk0_ZXYmk)

                            if rt == np.inf:
                                PYk1_ZXYmk = 1
                            else:
                                PYk1_ZXYmk = rt / (rt + 1)

                            uni = np.random.uniform(0,1)

                            YAct[K + k-1, t-1] = PYk1_ZXYmk > uni

                            lpv_y_new = lpv_y_act[0, int(YAct[K + k-1, t-1])]
                            lpx_y_new = lpx_y_act[0, int(YAct[K + k-1, t-1])]

                    for k in range(1, Knew + 1):
                        RAct[0, (K + k-1)] = beta.rvs(bAlpha + np.sum(YAct[K + k-1, :]), bBeta + T - np.sum(YAct[K + k-1, :]))

                    if wi > wburn:
                        lpy_v[wi - wburn - 1] = lpv_y_new
                        lpy_x[wi - wburn - 1] = lpx_y_new
                        lpy_xv[wi - wburn - 1] = lpx_y_new + lpv_y_new

            else:
                lpx_y_new = calc_px_training(X[n-1, :], YAct, ZAct[n-1, :], lda, eps, 1)
                lpv_y_new, _, _, _ = calc_pv_training(X, V, YAct, ZAct, sigmaU, sigmaV, phi, sigmaC, 0, 1)
                lpy_xv = lpx_y_new + lpv_y_new

            if Knew == 0:
                py_xzKnew_5 = lpy_xv
            else:
                py_xzKnew_5 = np.log(wsample) - np.log(1) + np.max(lpy_xv) - np.log(np.sum(1.0 / np.exp(lpy_xv - np.max(lpy_xv))))

            PKnew_XZY_5[0, Knew] = (py_xzKnew_5 + np.log(poisson.pmf(Knew, alpha / N)))

            Ysample[Knew] = YAct
            Rsample[Knew] = RAct

        lpdf_5 = PKnew_XZY_5 - np.max(PKnew_XZY_5)
        pdf_5 = np.exp(lpdf_5) / np.sum(np.exp(lpdf_5))

        bar = np.random.rand()
        #in MATLAB bar = 0.7792
        j = 1
        while np.sum(pdf_5.flatten()[:j]) < bar:
            j += 1
        Knew = j-1

        Znew = np.hstack((Z, np.zeros((N, Knew))))
        Znew[n-1, K:K + Knew] = 1

        Z = Znew.copy()
        Y = Ysample[Knew]
        R = Rsample[Knew]

        col_sums = np.sum(Z, axis=0)
        nodes2keep = np.nonzero(col_sums)[0]
        if nodes2keep.size > 0:
            Z = Z[:, nodes2keep]
            Y = Y[nodes2keep, :]
            R = R[0, nodes2keep].reshape(1,-1)
            K = Z.shape[1]
        else:
            Z = np.zeros((N, 1))
            Y = np.zeros((1, T))
            R = np.zeros((1, 1))
            K = Z.shape[1]

    Znew = Z
    Ynew = Y
    Rnew = R

    return Znew, Ynew, Rnew

def wood_ibp_learning_frontend(X, V, lda, eps, sigmaU, sigmaV, 
                               phi, sigmaC, alpha, bAlpha, bBeta, 
                               Kmax, wburn, wsample, stepNo, burnIn):
    
    initLatentNo = 1
    
    # 12 x 144
    N, T = X.shape

    # 12 x 1
    Znew = np.zeros((N, initLatentNo))

    # 1 x 1
    Rnew = beta.rvs(bAlpha, bBeta, size=(1, initLatentNo))

    # 144 x 12
    Ynew = np.array((np.random.rand(initLatentNo, T) < np.tile(Rnew.T, (1, T))), dtype=int)

    Zpost = np.empty((1, stepNo - burnIn), dtype=object)
    Ypost = np.empty((1, stepNo - burnIn), dtype=object)
    Rpost = np.empty((1, stepNo - burnIn), dtype=object)
    muCTpost = np.empty((1, stepNo - burnIn), dtype=object)
    sigmaCTpost = np.empty((1, stepNo - burnIn), dtype=object)

    for i in range(1, stepNo+1):
        print("#" * 50)
        print(f"iteration: {i-1}")
        print(f"Znew = \n{Znew}\n")
        print(f"Rnew = \n{Rnew}\n")

        Ynew = wood_make_gibbs_y_spatial(Ynew, X, V, Znew, Rnew, lda, eps, sigmaU, sigmaV, phi, sigmaC)
        
        Znew, Ynew, Rnew = wood_make_gibbs_z_spatial(Ynew, X, V, Znew, Rnew, lda, eps, sigmaU, sigmaV, phi, sigmaC, bAlpha, bBeta, alpha, 
                                                     Kmax, wburn, wsample)

        pi, muCTposti, SigmaCTpostInvi, sigmaCTposti = calc_pv_training(X, V, Ynew, Znew, sigmaU, sigmaV, phi, sigmaC, 0, 1)

        if i > burnIn:
            Zpost[0, i - burnIn - 1] = Znew
            Ypost[0, i - burnIn - 1] = Ynew
            Rpost[0, i - burnIn - 1] = Rnew
            muCTpost[0, i - burnIn - 1] = muCTposti
            sigmaCTpost[0, i - burnIn - 1] = sigmaCTposti

    return Zpost, Ypost, Rpost, muCTpost, sigmaCTpost, Kmax, wburn, wsample

Zpost, Ypost, Rpost, muCTpost, sigmaCTpost, Kmax, wburn, wsample = wood_ibp_learning_frontend(X[:, :trainingLength], V[:, :trainingLength, :], 
                                                                                                lda, eps, sigmaU, sigmaV, phi, sigmaC,alpha, 
                                                                                                bAlpha, bBeta, Kmax, wburn, wsample, stepNo, burnIn)

##################################################
iteration: 0
Znew = 
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

Rnew = 
[[0.44912526]]

##################################################
iteration: 1
Znew = 
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]

Rnew = 
[[0.80231439 0.37624982 0.28828397 0.2144073  0.15818236]]

##################################################
iteration: 2
Znew = 
[[1. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]

Rnew = 
[[0.90322068 0.12756125 0.56830803 0.31131225 0.27412053]]

##################################################
iteration: 3
Znew = 
[[1. 0. 1. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
